<a name="densenet"></a>
# Densenet

In this notebook we will be creating a pipeline using a pre-trained Densenet model which can be used for any image classification task. 

Densenet is a convolutional network where each layer is connected to all other layers that are deeper in the network
- The first layer is connected to the 2nd, 3rd, 4th etc.
- The second layer is connected to the 3rd, 4th, 5th etc.

Like this:

<img src="Images/densenet.png" alt="U-net Image"  align="middle"/>

For a detailed explanation of Densenet, check out the source of the image above, a paper by Gao Huang et al. 2018 called [Densely Connected Convolutional Networks](https://arxiv.org/pdf/1608.06993.pdf).

The cells below are set up to provide an exploration of the Keras implementation of Densenet.

### Importing Densenet from Keras

In [2]:
import warnings
warnings.filterwarnings('ignore')
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

### Creating the base pre-trained model

In [4]:
base_model = DenseNet121(weights='./nih/densenet.hdf5', include_top=False);

### Printing the model summary

In [23]:
# base_model.summary()

### Printing out the first five layers

In [9]:
layers_l = base_model.layers
print(f"There are total {len(layers_l)} layers")
print("First 5 layers")
layers_l[0:5]

There are total 427 layers
First 5 layers


### Printing out the last five layers

In [7]:
print("Last 5 layers")
layers_l[-6:-1]

Last 5 layers


### Getting the convolutional layers and print the first 5

In [12]:
conv2D_layers = [layer for layer in base_model.layers if str(type(layer)).find('Conv2D') > -1]
print("The first five conv2D layers")
conv2D_layers[0:5]

The first five conv2D layers


### Printing out the total number of convolutional layers

In [13]:
print(f"There are {len(conv2D_layers)} convolutional layers")

There are 120 convolutional layers


### Printing the number of channels in the input

In [14]:
print("The input has 3 channels")
base_model.input

The input has 3 channels


<tf.Tensor 'input_2:0' shape=(?, ?, ?, 3) dtype=float32>

### Printing the number of output channels

In [15]:
print("The output has 1024 channels")
x = base_model.output
x

The output has 1024 channels


<tf.Tensor 'relu_1/Relu:0' shape=(?, ?, ?, 1024) dtype=float32>

### Adding a global spatial average pooling layer

In [16]:
x_pool = GlobalAveragePooling2D()(x)
x_pool

<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(?, 1024) dtype=float32>

### Defining a set of five class labels to use as an example

In [17]:
labels = ['Emphysema', 
          'Hernia', 
          'Mass', 
          'Pneumonia',  
          'Edema']
n_classes = len(labels)
print(f"In this example, we want our model to identify {n_classes} classes")

In this example, we want our model to identify 5 classes


### Add a logistic layer the same size as the number of classes you're trying to predict

In [22]:
predictions = Dense(n_classes, activation="softmax")(x_pool)
print(f"Predictions have {n_classes} units, one for each class")
predictions

Predictions have 5 units, one for each class


<tf.Tensor 'dense_3/Softmax:0' shape=(?, 5) dtype=float32>

### Creating an updated model

In [20]:
model = Model(inputs=base_model.input, outputs=predictions)

### Compiling the model

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy')

# Thank you !!!